In [1]:
# Setup
import sys
from pathlib import Path
import importlib

project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

print(f"✅ Project root: {project_root}")

✅ Project root: D:\Academico\SENATI\octavo_ciclo\appBigData


In [2]:
# Imports
from sqlalchemy import create_engine, text
from config.config import Config
from src.logger_config import get_logger
import pandas as pd

# Recargar módulos
import src.transformer
importlib.reload(src.transformer)

from src.transformer import DataMartTransformer

logger = get_logger('notebook_transformacion')
logger.info("🚀 Iniciando transformación a modelo estrella")

✅ Configuración cargada correctamente
04:56:06 | notebook_transformacion | INFO | 🚀 Iniciando transformación a modelo estrella


In [3]:
# rear schema del Data Mart
"""
Ejecutar el script SQL para crear las tablas
"""
from sqlalchemy import text

engine_dm = create_engine(Config.get_dm_connection_string())

logger.info("🔨 Creando schema del Data Mart...")

# Leer script SQL
sql_file = project_root / 'sql' / 'create_datamart.sql'

with open(sql_file, 'r', encoding='utf-8') as f:
    sql_script = f.read()

# Ejecutar script
statements = []
for statement in sql_script.split(';'):
    statement = statement.strip()
    if statement and not statement.startswith('--') and statement.upper() != 'USE SAKILA_DW':
        statements.append(statement)

errores = 0
with engine_dm.connect() as conn:
    for i, statement in enumerate(statements):
        try:
            # Manejar DROP/CREATE VIEW
            if 'DROP VIEW' in statement.upper() or 'CREATE VIEW' in statement.upper():
                view_name = None
                if 'DROP VIEW IF EXISTS' in statement.upper():
                    view_name = statement.upper().split('DROP VIEW IF EXISTS')[1].split()[0].strip()
                    try:
                        conn.execute(text(f"DROP VIEW IF EXISTS {view_name}"))
                        conn.commit()
                    except:
                        pass
                    continue
                elif 'CREATE VIEW' in statement.upper():
                    conn.execute(text(statement))
                    conn.commit()
                    continue
            
            conn.execute(text(statement))
            conn.commit()
            
        except Exception as e:
            error_msg = str(e).lower()
            if 'already exists' not in error_msg and '1050' not in error_msg:
                logger.warning(f"⚠️  Statement {i+1}: {e}")
                errores += 1

logger.info(f"✅ Schema del Data Mart creado ({len(statements)} statements, {errores} errores)")

04:56:08 | notebook_transformacion | INFO | 🔨 Creando schema del Data Mart...
04:56:08 | notebook_transformacion | INFO | ✅ Schema del Data Mart creado (9 statements, 0 errores)


In [4]:
# Obtener ETL ID
engine_staging = create_engine(Config.get_staging_connection_string())

query_etl = text("""
    SELECT etl_id, proceso, fecha_fin, estado
    FROM etl_control
    WHERE estado = 'COMPLETADO'
    ORDER BY etl_id DESC
    LIMIT 1
""")

with engine_staging.connect() as conn:
    result = conn.execute(query_etl)
    ultimo_etl = result.fetchone()
    
    if ultimo_etl:
        etl_id = ultimo_etl[0]
        print(f"📝 Usando ETL ID: {etl_id}")
        print(f"   Proceso: {ultimo_etl[1]}")
    else:
        print("⚠️  No hay ETL previo, usando ID None")
        etl_id = None

📝 Usando ETL ID: 4
   Proceso: EXTRACCION_COMPLETA


In [5]:
# Inicializar transformador
transformer = DataMartTransformer(etl_id=etl_id)
logger.info("✅ Transformador inicializado")

04:56:16 | transformer | INFO | ✅ Transformador inicializado
04:56:16 | notebook_transformacion | INFO | ✅ Transformador inicializado


In [6]:
# Ejecutar transformación completa
"""
Poblar todas las dimensiones y tabla de hechos
"""
print("\n" + "="*80)
print("🔄 INICIANDO TRANSFORMACIÓN AL MODELO ESTRELLA")
print("="*80)

stats = transformer.ejecutar_transformacion_completa()

print("\n📊 Estadísticas de transformación:")
for key, value in stats.items():
    print(f"   {key}: {value:,}" if isinstance(value, int) else f"   {key}: {value}")



🔄 INICIANDO TRANSFORMACIÓN AL MODELO ESTRELLA
04:56:17 | transformer | INFO | ================================================================================
04:56:17 | transformer | INFO | 🚀 INICIO: TRANSFORMACION_DM
04:56:17 | transformer | INFO |    Creando modelo estrella
04:56:17 | transformer | INFO | ================================================================================
04:56:17 | transformer | INFO | 🕐 Poblando dim_tiempo (2005-01-01 a 2026-12-31)...
04:56:19 | transformer | INFO |    Tabla dim_tiempo limpiada
04:56:22 | transformer | INFO | ✅ dim_tiempo poblado: 8,035 registros
04:56:22 | transformer | INFO | 🎬 Poblando dim_film...
04:57:41 | transformer | INFO | ✅ dim_film: 1000 nuevos, 0 actualizados
04:57:41 | transformer | INFO | 📂 Poblando dim_categoria...
04:57:41 | transformer | INFO | ✅ dim_categoria: 16 registros
04:57:41 | transformer | INFO | 🏪 Poblando dim_tienda...
04:57:41 | transformer | INFO | ✅ dim_tienda: 2 registros
04:57:41 | transformer | INFO 

In [7]:
# Verificar dimensiones
"""
Contar registros en cada dimensión
"""
print("\n" + "="*80)
print("📋 DIMENSIONES POBLADAS")
print("="*80)

dimensiones = ['dim_tiempo', 'dim_film', 'dim_categoria', 'dim_tienda']

for dim in dimensiones:
    query = text(f"SELECT COUNT(*) as total FROM {dim}")
    with engine_dm.connect() as conn:
        result = conn.execute(query)
        total = result.fetchone()[0]
        print(f"✅ {dim}: {total:,} registros")

# Verificar tabla de hechos
query_fact = text("SELECT COUNT(*) as total FROM fact_ventas")
with engine_dm.connect() as conn:
    result = conn.execute(query_fact)
    total_fact = result.fetchone()[0]
    print(f"\n💰 fact_ventas: {total_fact:,} registros")


📋 DIMENSIONES POBLADAS
✅ dim_tiempo: 8,035 registros
✅ dim_film: 1,000 registros
✅ dim_categoria: 16 registros
✅ dim_tienda: 2 registros

💰 fact_ventas: 14,102 registros


In [8]:
# Ver muestra de datos - dim_film
"""
Examinar datos en dimensiones
"""
print("\n📊 Muestra de dim_film:")
query_film = "SELECT * FROM dim_film WHERE activo = TRUE LIMIT 10"
df_film = pd.read_sql(query_film, engine_dm)
display(df_film)


📊 Muestra de dim_film:


,film_sk,film_id,titulo,descripcion,anio_lanzamiento,duracion,clasificacion,tarifa_renta,costo_reemplazo,fecha_inicio,fecha_fin,version,activo
0,1,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,86,PG,0.99,20.99,2025-10-04 04:56:22,9999-12-31 23:59:59,1,1
1,2,2,ACE GOLDFINGER,A Astounding Epistle of a Database Administrat...,2006,48,G,4.99,12.99,2025-10-04 04:56:22,9999-12-31 23:59:59,1,1
2,3,3,ADAPTATION HOLES,A Astounding Reflection of a Lumberjack And a ...,2006,50,NC-17,2.99,18.99,2025-10-04 04:56:22,9999-12-31 23:59:59,1,1
3,4,4,AFFAIR PREJUDICE,A Fanciful Documentary of a Frisbee And a Lumb...,2006,117,G,2.99,26.99,2025-10-04 04:56:23,9999-12-31 23:59:59,1,1
4,5,5,AFRICAN EGG,A Fast-Paced Documentary of a Pastry Chef And ...,2006,130,G,2.99,22.99,2025-10-04 04:56:23,9999-12-31 23:59:59,1,1
5,6,6,AGENT TRUMAN,A Intrepid Panorama of a Robot And a Boy who m...,2006,169,PG,2.99,17.99,2025-10-04 04:56:23,9999-12-31 23:59:59,1,1
6,7,7,AIRPLANE SIERRA,A Touching Saga of a Hunter And a Butler who m...,2006,62,PG-13,4.99,28.99,2025-10-04 04:56:23,9999-12-31 23:59:59,1,1
7,8,8,AIRPORT POLLOCK,A Epic Tale of a Moose And a Girl who must Con...,2006,54,R,4.99,15.99,2025-10-04 04:56:23,9999-12-31 23:59:59,1,1
8,9,9,ALABAMA DEVIL,A Thoughtful Panorama of a Database Administra...,2006,114,PG-13,2.99,21.99,2025-10-04 04:56:23,9999-12-31 23:59:59,1,1
9,10,10,ALADDIN CALENDAR,A Action-Packed Tale of a Man And a Lumberjack...,2006,63,NC-17,4.99,24.99,2025-10-04 04:56:23,9999-12-31 23:59:59,1,1


In [9]:
# Ver muestra de datos - fact_ventas
"""
Ver datos agregados en la tabla de hechos
"""
print("\n📊 Muestra de fact_ventas:")
query_fact = """
    SELECT 
        fv.venta_id,
        t.fecha,
        f.titulo,
        c.nombre_categoria,
        ti.nombre_tienda,
        fv.cantidad_rentas,
        fv.monto_total
    FROM fact_ventas fv
    JOIN dim_tiempo t ON fv.fecha_id = t.fecha_id
    JOIN dim_film f ON fv.film_sk = f.film_sk
    JOIN dim_categoria c ON fv.categoria_sk = c.categoria_sk
    JOIN dim_tienda ti ON fv.tienda_sk = ti.tienda_sk
    ORDER BY fv.monto_total DESC
    LIMIT 20
"""
df_fact = pd.read_sql(query_fact, engine_dm)
display(df_fact)



📊 Muestra de fact_ventas:


,venta_id,fecha,titulo,nombre_categoria,nombre_tienda,cantidad_rentas,monto_total
0,11847,2005-08-20,CALIFORNIA BIRDS,Sports,Tienda 2,3,22.97
1,10353,2005-08-17,EASY GLADIATOR,Action,Tienda 1,3,22.97
2,7835,2005-07-29,SCORPION APOLLO,Drama,Tienda 1,2,21.98
3,13815,2005-08-23,SCORPION APOLLO,Drama,Tienda 2,3,21.97
4,10975,2005-08-18,JERSEY SASSY,Children,Tienda 2,3,21.97
5,8778,2005-07-31,KISS GLORY,Foreign,Tienda 2,3,21.97
6,7208,2005-07-28,MOONSHINE CABIN,Games,Tienda 2,3,21.97
7,7002,2005-07-28,DAISY MENAGERIE,Sci-Fi,Tienda 2,3,21.97
8,6604,2005-07-27,INNOCENT USUAL,Foreign,Tienda 2,2,20.98
9,9028,2005-07-31,TYCOON GATHERING,Games,Tienda 1,2,20.98


In [10]:
# Probar vistas analíticas
"""
Ejecutar queries de negocio usando las vistas
"""
print("\n" + "="*80)
print("📈 QUERIES ANALÍTICAS - VISTAS PREDEFINIDAS")
print("="*80)

# Vista 1: Top films por categoría
print("\n🎬 Top 5 películas más rentadas por categoría:")
query_top = """
    SELECT 
        nombre_categoria,
        titulo,
        total_rentas,
        ingresos_totales
    FROM v_top_films_categoria
    LIMIT 20
"""
df_top = pd.read_sql(query_top, engine_dm)
display(df_top)


📈 QUERIES ANALÍTICAS - VISTAS PREDEFINIDAS

🎬 Top 5 películas más rentadas por categoría:


,nombre_categoria,titulo,total_rentas,ingresos_totales
0,Action,SUSPECTS QUILLS,30.0,133.70
1,Action,RUGRATS SHAKESPEARE,30.0,70.70
2,Action,STORY SIDE,28.0,39.72
3,Action,TRIP NEWTON,28.0,145.72
4,Action,HANDICAP BOONDOCK,28.0,63.72
5,Action,PRIMARY GLASS,27.0,36.73
6,Action,FANTASY TROOPERS,26.0,45.74
7,Action,STAGECOACH ARMAGEDDON,26.0,154.74
8,Action,CLUELESS BUCKET,25.0,112.75
9,Action,HILLS NEIGHBORS,25.0,72.75


In [11]:
# Vista - Performance por categoría
print("\n📂 Performance por categoría:")
query_perf = "SELECT * FROM v_performance_categoria ORDER BY ingresos_totales DESC"
df_perf = pd.read_sql(query_perf, engine_dm)
display(df_perf)



📂 Performance por categoría:


,nombre_categoria,total_peliculas,total_rentas,ingresos_totales,precio_promedio,dias_renta_promedio
0,Sports,73,1179.0,5314.21,4.505322,5.117007
1,Sci-Fi,59,1101.0,4756.98,4.338969,5.017173
2,Animation,64,1166.0,4656.30,3.995699,4.789695
3,Drama,61,1060.0,4587.39,4.302728,4.841923
4,Comedy,56,941.0,4383.58,4.636953,5.088614
5,Action,61,1112.0,4375.85,3.970062,4.975644
6,New,60,940.0,4351.62,4.609659,4.919708
7,Games,58,969.0,4281.33,4.425706,5.150612
8,Foreign,67,1033.0,4270.67,4.112745,4.988412
9,Family,67,1096.0,4226.07,3.859442,4.989163


In [12]:
# Vista - Ventas mensuales por tienda
print("\n🏪 Ventas mensuales por tienda:")
query_mensual = """
    SELECT * FROM v_ventas_mensuales_tienda
    ORDER BY anio DESC, mes DESC, ingresos_totales DESC
    LIMIT 20
"""
df_mensual = pd.read_sql(query_mensual, engine_dm)
display(df_mensual)


🏪 Ventas mensuales por tienda:


,anio,mes,mes_nombre,nombre_tienda,ciudad,total_rentas,ingresos_totales,ticket_promedio
0,2006,2,February,Tienda 2,Woodridge,90.0,271.08,2.989775
1,2006,2,February,Tienda 1,Lethbridge,92.0,243.10,2.705349
2,2005,8,August,Tienda 2,Woodridge,2885.0,12136.15,4.237526
3,2005,8,August,Tienda 1,Lethbridge,2801.0,11933.99,4.250079
4,2005,7,July,Tienda 1,Lethbridge,3334.0,14308.66,4.293958
5,2005,7,July,Tienda 2,Woodridge,3375.0,14060.25,4.159501
6,2005,6,June,Tienda 2,Woodridge,1190.0,4895.10,4.147193
7,2005,6,June,Tienda 1,Lethbridge,1121.0,4734.79,4.201840
8,2005,5,May,Tienda 1,Lethbridge,575.0,2459.25,4.249563
9,2005,5,May,Tienda 2,Woodridge,581.0,2364.19,4.071839


In [13]:
# Vista - Resumen ejecutivo
print("\n📊 Resumen ejecutivo mensual:")
query_resumen = "SELECT * FROM v_resumen_ejecutivo ORDER BY anio DESC, mes DESC LIMIT 12"
df_resumen = pd.read_sql(query_resumen, engine_dm)
display(df_resumen)


📊 Resumen ejecutivo mensual:


,anio,mes,peliculas_rentadas,tiendas_activas,total_rentas,ingresos_totales,ticket_promedio,rentas_por_tienda
0,2006,2,168,2,182.0,514.18,2.850000,91.0
1,2005,8,958,2,5686.0,24070.14,4.243706,2843.0
2,2005,7,958,2,6709.0,28368.91,4.225565,3354.5
3,2005,6,900,2,2311.0,9629.89,4.173413,1155.5
4,2005,5,686,2,1156.0,4823.44,4.159820,578.0


In [14]:
# Query personalizado - Top tiendas
"""
Query ad-hoc para análisis específico
"""
print("\n🏆 Top tiendas por ingresos:")
query_tiendas = """
    SELECT 
        ti.nombre_tienda,
        ti.ciudad,
        ti.pais,
        SUM(fv.cantidad_rentas) as total_rentas,
        SUM(fv.monto_total) as ingresos_totales,
        AVG(fv.monto_promedio) as ticket_promedio
    FROM fact_ventas fv
    JOIN dim_tienda ti ON fv.tienda_sk = ti.tienda_sk
    WHERE ti.activo = TRUE
    GROUP BY ti.nombre_tienda, ti.ciudad, ti.pais
    ORDER BY ingresos_totales DESC
"""
df_tiendas = pd.read_sql(query_tiendas, engine_dm)
display(df_tiendas)


🏆 Top tiendas por ingresos:


,nombre_tienda,ciudad,pais,total_rentas,ingresos_totales,ticket_promedio
0,Tienda 2,Woodridge,Australia,8121.0,33726.77,4.163059
1,Tienda 1,Lethbridge,Canada,7923.0,33679.79,4.241875


In [15]:
# Comparación de performance: OLTP vs OLAP
"""
Comparar tiempo de ejecución entre Sakila y Data Mart
"""
import time

print("\n" + "="*80)
print("⚡ COMPARACIÓN DE PERFORMANCE: OLTP vs OLAP")
print("="*80)

engine_sakila = create_engine(Config.get_sakila_connection_string())

# Query compleja en Sakila (OLTP - normalizado)
query_oltp = """
    SELECT 
        c.name as categoria,
        f.title as pelicula,
        COUNT(r.rental_id) as total_rentas,
        SUM(p.amount) as ingresos
    FROM rental r
    JOIN inventory i ON r.inventory_id = i.inventory_id
    JOIN film f ON i.film_id = f.film_id
    JOIN film_category fc ON f.film_id = fc.film_id
    JOIN category c ON fc.category_id = c.category_id
    LEFT JOIN payment p ON r.rental_id = p.rental_id
    GROUP BY c.name, f.title
    ORDER BY ingresos DESC
    LIMIT 20
"""

# Mismo query en Data Mart (OLAP - desnormalizado)
query_olap = """
    SELECT 
        c.nombre_categoria as categoria,
        f.titulo as pelicula,
        SUM(fv.cantidad_rentas) as total_rentas,
        SUM(fv.monto_total) as ingresos
    FROM fact_ventas fv
    JOIN dim_film f ON fv.film_sk = f.film_sk
    JOIN dim_categoria c ON fv.categoria_sk = c.categoria_sk
    GROUP BY c.nombre_categoria, f.titulo
    ORDER BY ingresos DESC
    LIMIT 20
"""

# Ejecutar y medir tiempos
print("\n🔍 Ejecutando query en OLTP (Sakila normalizado)...")
start_oltp = time.time()
df_oltp = pd.read_sql(query_oltp, engine_sakila)
time_oltp = time.time() - start_oltp

print(f"⏱️  Tiempo OLTP: {time_oltp:.3f} segundos")

print("\n🔍 Ejecutando query en OLAP (Data Mart desnormalizado)...")
start_olap = time.time()
df_olap = pd.read_sql(query_olap, engine_dm)
time_olap = time.time() - start_olap

print(f"⏱️  Tiempo OLAP: {time_olap:.3f} segundos")

mejora = ((time_oltp - time_olap) / time_oltp * 100)
print(f"\n🚀 Mejora de performance: {mejora:.1f}% más rápido")
print(f"   ({time_oltp/time_olap:.1f}x más rápido)")


⚡ COMPARACIÓN DE PERFORMANCE: OLTP vs OLAP

🔍 Ejecutando query en OLTP (Sakila normalizado)...
⏱️  Tiempo OLTP: 0.188 segundos

🔍 Ejecutando query en OLAP (Data Mart desnormalizado)...
⏱️  Tiempo OLAP: 0.071 segundos

🚀 Mejora de performance: 61.9% más rápido
   (2.6x más rápido)


In [16]:
# Cerrar conexiones
transformer.cerrar_conexiones()
logger.info("✅ Transformación completada")

print("\n" + "="*80)
print("🎉 ¡MODELO ESTRELLA COMPLETADO!")
print("="*80)
print("\n✅ Data Mart listo para consumo en Power BI")
print("📌 Siguiente paso: Conectar Power BI a sakila_dw")
print("\n📊 Tablas disponibles:")
print("   - dim_tiempo, dim_film, dim_categoria, dim_tienda")
print("   - fact_ventas")
print("\n📈 Vistas analíticas:")
print("   - v_top_films_categoria")
print("   - v_ventas_mensuales_tienda")
print("   - v_performance_categoria")
print("   - v_resumen_ejecutivo")

04:59:40 | transformer | INFO | 🔌 Conexiones cerradas
04:59:40 | notebook_transformacion | INFO | ✅ Transformación completada

🎉 ¡MODELO ESTRELLA COMPLETADO!

✅ Data Mart listo para consumo en Power BI
📌 Siguiente paso: Conectar Power BI a sakila_dw

📊 Tablas disponibles:
   - dim_tiempo, dim_film, dim_categoria, dim_tienda
   - fact_ventas

📈 Vistas analíticas:
   - v_top_films_categoria
   - v_ventas_mensuales_tienda
   - v_performance_categoria
   - v_resumen_ejecutivo
